In [146]:
import numpy as np

In [1]:
txt_test = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
..........."""

In [2]:
with open("input/21", "r") as fp:
    txt = fp.read()[:-1]

In [147]:
def process_input(ttt):
    lines = ttt.split('\n')
    l0 = len(lines)
    l1 = len(lines[0])
    M = np.zeros((l0, l1))
    
    start = None
    for idx, row in enumerate(lines):
        for jdx, v in enumerate(row):
            if v == "#":
                M[idx, jdx] = 1
            elif v == "S":
                start = (idx, jdx)
                
    return M, start

# Part 1

In [728]:
def update_move(M0, lst):
    
    pos1 = np.concatenate([
        lst + np.array([0, 1]), 
        lst + np.array([0, -1]), 
        lst + np.array([-1, 0]),
        lst + np.array([1, 0])
                   ])
    # Remove invalid positions
    pos1 = pos1[(pos1[:, 0] >= 0) & (pos1[:, 1] >= 0) & (pos1[:, 0] < len(M0)) & (pos1[:, 1] < len(M0[0]))]
    pos1 = pos1[M0[pos1[:, 0], pos1[:, 1]] == 0]
    
    M1 = M0.copy()
    M1[pos1[:,0], pos1[:, 1]] = 2
    
                
    return np.array(np.where(M1 == 2)).T

In [19]:
M0, start = process_input(txt)

M0.shape, start

((131, 131), (65, 65))

In [24]:
positions = np.array([start])

for idx in range(1, 65):
    positions = update_move(M0, positions)
    tot = len(positions)

print("Step {}: \t {}".format(idx, tot))

Step 64: 	 3820


# Part 2: Infinite map

## FX

In [ ]:
S = 26501365

In [748]:
def replicate(M0, k0, k1):
    """
    Replicate the matrix M0 
    - k0 times vertically
    - k1 times horizontally
    """
    M1 = np.concatenate([M0 for _ in range(k0)]).T
    return np.concatenate([M1 for _ in range(k1)]).T
    
    

def get_info(X, i=2):
    # i: where it starts repeating
    
    period = sum(X[:, i+1] == 0) - sum(X[:, i] == 0)
    v_start = sum(X[:, i] == 0) - i *period
    # A cycle starts at t = period * i + v_start
    
    left = period * i + v_start
    for j in range(len(X)):
        if (X[left + j, i] == X[left + j+2, i]) & (X[left + j+1, i] == X[left + j+3, i]):
            break
    
    return (period, v_start, X[left: left+j, i], X[left+j: left+j+4, i])
        
    

In [359]:
def count_last_tiles(F, d):
    period = F[0]
    lag = F[1] # Value
    stability = F[2] # array
    loop = F[3]
    
    k_full = (d - lag - len(stability)) // period
    k_partial = max(0, ((d - lag)) // period +1 - k_full)
    
    
    even = k_full // 2
    odd = (k_full+1) // 2
    
    # GOOOD 
    d0 = lag + period * k_full
    tot = 0
    for i in range(1, k_partial):
        tot += stability[d - d0 - i * period]
        
    return ( odd, even, k_partial, odd * loop[(d - len(stability) - lag + 1 ) % 2],even * loop[(d - len(stability)  - lag)%2], tot)
    

In [532]:
def get_establishment(seq):
    
    for i, v in enumerate(seq):
        if seq[i+2] == v:
            if seq[i+1] == seq[i+3]:
                return i, seq[i:i+4]
            
        

In [621]:
def extract_quadrant_info(X, locs, s=3):
    
    # Behavior of 0x0
    seq0 = X[:, locs[0,0]]
    t0, a0 = get_establishment(seq0)
    
    F_1 = get_info(X[:, locs[0]], s)
    F_2 = get_info(X[:, locs[:, 0]], s)
    
    k = len(locs)
    
    d0 = np.arange(1, k)
    lll = locs[d0, d0]    
    F_d1 = get_info(X[:, lll], s)

    lll = locs[d0-1, d0]
    F_d2 = get_info(X[:, lll], s)

    return t0, a0, F_1, F_2, F_d1, F_d2

In [671]:
def get_count_diags(idx, t0, a0, F_d1, F_d2):
    """
    
    Complicated formula to compute the diagonal values.
    Note: not valid until 3 frequencies
    """
    v0 = a0[(idx-t0) % 2]
    v1 = a0[(1+idx-t0) % 2]

    a, b, c, t1, t2, left = count_last_tiles(F_d1, idx)
    diag = a + b + 1

    # Diag 1
    vals_out = (diag * 2 + 1) * left
    vals_in = diag * diag * v0
    P1 = vals_in + vals_out

    a, b, c, t1, t2, left = count_last_tiles(F_d2, idx)
    diag = a + b 

    vals_out = ((1+diag) * 2) * left
    vals_in = diag * (diag+1) * v1
    P2 = vals_in + vals_out

    return P1 + P2

In [661]:
def get_count_linear(idx, F):
    a, b, c, t1, t2, left = count_last_tiles(F, idx)
    return t1+t2+left
    

# Part 2 Clean

In [738]:
def update_move_v2(M0, lst):
    """Update the matrix M0
    """
    M0[M0 == 2] = 3
    
    pos1 = np.concatenate([
        lst + np.array([0, 1]), 
        lst + np.array([0, -1]), 
        lst + np.array([-1, 0]),
        lst + np.array([1, 0])
                   ])
    # Remove invalid positions
    pos1 = pos1[(pos1[:, 0] >= 0) & (pos1[:, 1] >= 0) & (pos1[:, 0] < len(M0)) & (pos1[:, 1] < len(M0[0]))]
    pos1 = pos1[M0[pos1[:, 0], pos1[:, 1]] == 0]
    
    
    M0[pos1[:,0], pos1[:, 1]] = 2
    
                
    return np.array(np.where(M0 == 2)).T

In [745]:
M0, start = process_input(txt)

l0, l1 = M0.shape
M0.shape, start

((131, 131), (65, 65))

In [746]:
k = 6 # minimum necessary
Mx = replicate(M0, k, k)
locs = np.arange(k*k).reshape((k, k))

print(len(Mx) + len(Mx[0]))

X_lst = []
for M1 in [Mx, Mx[::-1], Mx[::-1,::-1], Mx[:,::-1]]:
    
    M1a = M1.copy()
    M1b = M1.copy()
    
    print('-'*40)
    positions = np.array([(start[0], start[1])])

    lst = []
    for idx in range(1, len(M1) + len(M1[0])):
        if idx % 10 == 0:
            print(idx, end=", ")
        positions = update_move_v2(M1a, positions)
        
        
        tmp = []
        for i0 in range(k):
            for i1 in range(k):
                tot = (M1a[i0*l0:(i0+1)*l0, i1*l1:(i1+1)*l1] >= 2).sum()
                
                
                tmp.append(tot)

        lst.append(tmp)
        # Exchange matrices
        M1a, M1b = M1b, M1a 
    
        
    X_lst.append(np.array(lst))


1572
----------------------------------------
10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, ----------------------------------------
10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180

### Compute results

In [747]:
idx = 5000-1
idx = 26501365 - 1

CNT = 0
CNT1 = 0
for X in X_lst:
    t0, a0, F_1, F_2, F_d1, F_d2 = extract_quadrant_info(X, locs)
    V1 = get_count_linear(idx, F_1)
    V2 = get_count_linear(idx, F_2)
    VD = get_count_diags(idx, t0, a0, F_d1, F_d2)
    print(V1, V2)
    
    CNT += VD
    CNT1 += V1 + V2

v0 = a0[(idx-t0) % 2]

print("Part 2: ", v0 + CNT + CNT1//2)

1563069039 1563069055
1563069039 1563069043
1563069059 1563069043
1563069059 1563069055
Part 2:  632421652138917
